In [ ]:
# default_exp distance

# Distance
> Distance calculation

Wiki on [k-d-tree](https://en.wikipedia.org/wiki/K-d_tree)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import numpy as np
import torch
import plotly.graph_objects as go
import plotly.express as px
import timeit
import typing
import itertools
import pandas as pd
import math

## Direct pairwise distances

Example positions and distances

In [ ]:
positions = torch.tensor([[0,0],
                          [1,1],
                          [2,2]], dtype=torch.float)
correct_distances = torch.tensor([2., 8., 2.], dtype=torch.float)

In [ ]:
#export
class PairwiseDistance:
    def __call__(self, x:torch.Tensor, method:str='torch'):
        return getattr(self, f'{method}_pairwise_distance')(x)

In [ ]:
pdist = PairwiseDistance()

Using the Gram matrix to compute pairwise distances: [stackexchange](https://math.stackexchange.com/questions/2240429/pairwise-distance-matrix)

In [ ]:
#export
def pairwise_dist_gram(x:torch.Tensor, y:torch.Tensor,
                       flat:bool=True) -> torch.Tensor:
    
    nx = x.size()[0]
    ny = y.size()[0]
    
    x2 = torch.mm(x, x.t())
    y2 = torch.mm(y, y.t())
    xy = torch.mm(x, y.t())
    
    x2 = x2.diag().unsqueeze(0).expand(ny,-1)
    y2 = y2.diag().unsqueeze(0).expand(nx,-1)
    d = x2.t() + y2 - 2*xy
    if flat:
        ix = torch.triu_indices(nx,ny,offset=1) #list(zip(*list(itertools.combinations(range(len(positions)), r=2))))
        return d[ix[0],ix[1]]
    return d

In [ ]:
%%time
pairwise_dist_gram(positions, positions)

In [ ]:
%%timeit
pairwise_dist_gram(positions, positions)

In [ ]:
#export
def stackoverflow_pairwise_distance(self, x:torch.Tensor):
    return pairwise_dist_gram(x, x)

PairwiseDistance.stackoverflow_pairwise_distance = stackoverflow_pairwise_distance

In [ ]:
pdist(positions, method='stackoverflow')

In [ ]:
%timeit pdist(positions, method='stackoverflow')

In [ ]:
assert torch.isclose(correct_distances, 
              pdist(positions, method='stackoverflow')).all()

Using pre-existing implementation in pytorch: `torch.nn.PairwiseDistance`

In [ ]:
#export
def torch_pairwise_distance(self, x:torch.Tensor):
    nx = ny = x.size()[0]
    pdist = torch.nn.PairwiseDistance(p=2, keepdim=True)
    ix = torch.triu_indices(nx,ny,offset=1)
    return pdist(x[ix[0],:], x[ix[1],:]).squeeze(1) ** 2

PairwiseDistance.torch_pairwise_distance = torch_pairwise_distance

In [ ]:
pdist(positions, method='torch')

In [ ]:
%timeit pdist(positions, method='torch')

In [ ]:
assert torch.isclose(correct_distances, 
              pdist(positions, method='torch')).all()

**Testing the scaling behavior**

In [ ]:
#export
def measure_execution_time(fun:typing.Callable, args, kwargs, 
                           repetitions:int=3, number:int=21):
    return (timeit.Timer(lambda: fun(*args, **kwargs))
            .repeat(repeat=repetitions, number=number))


def get_time_stats(fun:typing.Callable, n:int, d:int, 
                   methods:typing.List[str]=['torch', 'stackoverflow'],
                   reps:int=7, number:int=21):
    
    timing_stats = []
    for method, _n, _d in itertools.product(methods, n, d):
        positions = torch.randn(_n, _d)
        timing_stats.append({
            'n':_n,
            'd':_d,
            'method': method,
            'ts': measure_execution_time(fun, args=(positions,), 
                                         kwargs=dict(method=method),
                                         repetitions=reps,
                                         number=number)
        })
    return pd.DataFrame(timing_stats)

In [ ]:
# number of positions
n = [10, 100, 1000]
# number of dimensions
d = [2, 3, 10]

In [ ]:
%%time
timing_stats = get_time_stats(pdist, n, d)

In [ ]:
timing_stats.head()

In [ ]:
timing_stats_e = timing_stats.explode('ts')

In [ ]:
n_filter = 1000
d_filter = 3

In [ ]:
mask_n = timing_stats['n'] == n_filter
mask_d = timing_stats['d'] == d_filter

In [ ]:
px.scatter(timing_stats_e.loc[mask_d], x='n', y='ts', color='method',
           title=f'Performance @ {d_filter} dims')

In [ ]:
px.scatter(timing_stats_e.loc[mask_n], x='d', y='ts', color='method',
           title=f'Performance @ {n_filter} samples')

The following is based on stackexchange:
- [answer1](https://math.stackexchange.com/questions/2134011/conversion-of-upper-triangle-linear-index-from-index-on-symmetrical-array)
- [answer2](https://stackoverflow.com/questions/19143657/linear-indexing-in-symmetric-matrices)

In [ ]:
#export
class DistanceMatrixIndexMapper:
    
    ix_fun = lambda self, i, j, n: int(n*i -.5*i*(i+1) + j)
    i_fun = lambda self, x, n: int(math.floor((2*n+1 - math.sqrt((2*n+1)**2 - 8*x))*.5))
    j_fun = lambda self, x, i, n: int(x - (n*i -.5*i*(i+1)))

    def __init__(self,num_pos:int):
        self.num_pos = num_pos
    
    def __call__(self,i:int, direction:str, j:int=None):
        return getattr(self, direction)(i,j)
    
    def brute_force_square2flat(self, i:int, j:int):
        if i > j: i, j = j, i
        self._check_ix_map()
        return self.ix_map[i,j]
        
    def _check_ix_map(self):
        if not hasattr(self, 'ix_map'):
            self.ix_map = {ix: i for i, ix in enumerate(itertools.combinations_with_replacement(range(self.num_pos), r=2))}
        
    def brute_force_flat2square(self, ix:int, j:int=None):
        self._check_ix_map_inv()
        return self.ix_map_inv[ix]
    
    def _check_ix_map_inv(self):
        self._check_ix_map()
        if not hasattr(self, 'ix_map_inv'):
            self.ix_map_inv = {i: ix for ix, i in self.ix_map.items()}
    
    def analytical_square2flat(self, i:int, j:int):
        if i > j: i, j = j, i
        return self.ix_fun(i, j, self.num_pos)
    
    def analytical_flat2square(self, ix:int, j:int=None):
        return (i := self.i_fun(ix, self.num_pos)), (self.j_fun(ix, i, self.num_pos))


In [ ]:
i = 0
j = 1
ix = 1

num_pos = len(positions)

In [ ]:
dmap = DistanceMatrixIndexMapper(num_pos)

Square $(i,j) \rightarrow ix$ flat

In [ ]:
dmap(i,'brute_force_square2flat', j)

In [ ]:
dmap(i,'analytical_square2flat', j)

In [ ]:
#hide
%timeit dmap(i,'brute_force_square2flat', j)

In [ ]:
#hide
%timeit dmap(i,'analytical_square2flat', j)

Flat $ix \rightarrow (i,j)$ square

In [ ]:
dmap(ix,'brute_force_flat2square')

In [ ]:
dmap(ix,'analytical_flat2square')

In [ ]:
#hide
%timeit dmap(ix,'brute_force_flat2square')

In [ ]:
#hide
%timeit dmap(ix,'analytical_flat2square')

So the analytical version is quite a bit slower, but on the other hand does not have a memory scaling to consider, in contrast to the brute force variant

Performing sanity checks

In [ ]:
#hide
%%time
df = pd.DataFrame([[ix, i, j] for ix, (i,j) in enumerate(zip(*np.triu_indices(num_pos, k=0)))],
                  columns=['ix', 'i', 'j'])

for method in ['brute_force', 'analytical']:
    df_test = pd.DataFrame(columns=['ix','i','j'], dtype=int)
    df_test['i'], df_test['j'] = list(zip(*df.apply(lambda x: dmap(x['ix'], 'analytical_flat2square'), axis=1).values))
    df_test['ix'] = df.apply(lambda x: dmap(x['i'], 'analytical_square2flat', x['j']), axis=1).values

    assert df_test.equals(df), f'{method}: failed! `df` {df}\n!=\n`df_test` {df_test}'

Visualising the correct and the calculated (`-fun`) maps

In [ ]:
go.Figure(data=[
    go.Scatter(x=df['ix'], y=df['ix'], mode='markers', name='ix'),
    go.Scatter(x=df_test['ix'], y=df_test['ix'], mode='markers', 
               marker_symbol='x-open', name='ix-fun'),
],
         layout=go.Layout(title=f'Positions: {num_pos}', 
                          xaxis_title='ix', yaxis_title='calculated'))

In [ ]:
go.Figure(data=[
    go.Scatter(x=df['ix'], y=df['i'], mode='markers', name='i'),
    go.Scatter(x=df['ix'], y=df['j'], mode='markers', name='j'),
    go.Scatter(x=df_test['ix'], y=df_test['i'], mode='markers', 
               marker_symbol='x-open', name='i-fun'),
    go.Scatter(x=df_test['ix'], y=df_test['j'], mode='markers', 
               marker_symbol='x-open', name='j-fun'),
],
         layout=go.Layout(title=f'Positions: {num_pos}', 
                          xaxis_title='ix', yaxis_title='calculated'))